In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [2]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle

Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
     |████████████████████████████████| 1.5MB 8.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.9-py3-none-any.whl size=73265 sha256=216dbdac7b12aa4a19bf95aa1a9b0402324737e50a8f1ad68c4ac3ec221313eb
  Stored in directory: /root/.cache/pip/wheels/9d/50/3d/2644504bb1e8c782f3fef5984f03d76fc4a74698fdec128b29
Successfully built kaggle


In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c ai-project-life-environment

  0% 0.00/5.42k [00:00<?, ?B/s]
100% 5.42k/5.42k [00:00<00:00, 8.45MB/s]


In [5]:
!unzip ai-project-life-environment.zip

Archive:  ai-project-life-environment.zip
  inflating: submit sample.csv       
  inflating: test data.csv           
  inflating: train data.csv          


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [8]:
train = pd.read_csv('train data.csv',header=None,skiprows=1)
test = pd.read_csv('test data.csv',header=None,skiprows=1)
submit = pd.read_csv('submit sample.csv')

train[:5]

,0,1,2,3,4,5,6,7,8
0,201101,27.7,27.9,29.4,21.2,30.5,35.1,48.8,61.0
1,201102,26.9,28.1,29.1,21.9,31.3,35.0,50.1,61.6
2,201103,30.3,26.9,30.5,18.6,27.5,35.3,44.1,58.8
3,201104,29.6,27.0,27.9,19.3,26.3,34.2,47.0,55.0
4,201105,28.2,27.1,32.6,22.5,32.4,38.0,52.0,60.3


In [9]:
train[0] = train[0]%10000/100
test[0] = test[0]%10000/100

train[:5]

,0,1,2,3,4,5,6,7,8
0,11.01,27.7,27.9,29.4,21.2,30.5,35.1,48.8,61.0
1,11.02,26.9,28.1,29.1,21.9,31.3,35.0,50.1,61.6
2,11.03,30.3,26.9,30.5,18.6,27.5,35.3,44.1,58.8
3,11.04,29.6,27.0,27.9,19.3,26.3,34.2,47.0,55.0
4,11.05,28.2,27.1,32.6,22.5,32.4,38.0,52.0,60.3


In [10]:
x_train = train.loc[:,0:7]
y_train = train.loc[:,8]

x_test = test.loc[:,0:7]

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train_std = sc.fit_transform(x_train)
x_test_std = sc.fit_transform(x_test)

---
#KNN
---

In [29]:
from sklearn.neighbors import KNeighborsRegressor
KNN = KNeighborsRegressor(n_neighbors = 5, p = 2)

KNN.fit(x_train,y_train)
train_pred = KNN.predict(x_train)
print(mean_squared_error(y_train,train_pred))

3.152337454545454


In [19]:
|# ver std
KNN.fit(x_train_std,y_train)
train_pred = KNN.predict(x_train_std)
print(mean_squared_error(y_train,train_pred))

0.0


In [30]:
test_pred = KNN.predict(x_test)

for i in range(len(test_pred)):
  submit['Expected'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
!kaggle competitions submit -c ai-project-life-environment -f submission.csv -m "KNN_5_"

100% 411/411 [00:04<00:00, 101B/s]
Successfully submitted to 인공지능 텀프로젝트

#### no_std_ k = 5 weights= 'distance'/ 1.643
#### std_ k = 5 weights= 'distance'/ 1.75
#### no_std_ k = 7 weights= 'distance'/ 1.66
#### no_std_ k = 2 weights= 'distance'/ 1.88
#### no_std_ k = 5 / 1.644

---
#Logistic Regression
---

In [55]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C = 0.1,random_state = 1,solver = 'saga')


y_train_encode = y_train.astype(int)
logit.fit(x_train,y_train_encode)
train_pred = logit.predict(x_train)
print(mean_squared_error(y_train_encode,train_pred))

5.352727272727273


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [56]:
test_pred = logit.predict(x_test)

for i in range(len(test_pred)):
  submit['Expected'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [57]:
!kaggle competitions submit -c ai-project-life-environment -f submission.csv -m "logit_0.1"

100% 202/202 [00:03<00:00, 53.1B/s]
Successfully submitted to 인공지능 텀프로젝트

---
#### 100.0 / 1.89
#### 0.1 / 1.88

---
#Decision Tree
---

In [69]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state = 1,max_depth= 7)

dtr.fit(x_train,y_train)
train_predict = dtr.predict(x_train)
print(mean_squared_error(y_train,train_predict))

1.4719141667460711


In [70]:
test_pred = dtr.predict(x_test)

for i in range(len(test_pred)):
  submit['Expected'][i] = test_pred[i]

submit.to_csv('submission.csv',header=True,index=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [71]:
!kaggle competitions submit -c ai-project-life-environment -f submission.csv -m "dtr"

100% 388/388 [00:03<00:00, 115B/s]
Successfully submitted to 인공지능 텀프로젝트

#### (random_state = 1,max_features = "sqrt") / 4.4
#### (random_state = 1,max_features = "sqrt") / 3.01
#### (random_state = 1,max_depth= 5) / 2.61
#### (random_state = 1,max_depth= 7) / 3.04